In [1]:
!pip install mtcnn
!kaggle datasets download vishesh1412/celebrity-face-image-dataset

import os

import cv2
from mtcnn import MTCNN
from google.colab.patches import cv2_imshow



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 54.0 MB/s eta 0:00:00
Dataset URL: https://www.kaggle.com/datasets/vishesh1412/celebrity-face-image-dataset
License(s): CC0-1.0
 93% 49.0M/52.9M [00:01<00:00, 40.0MB/s]
100% 52.9M/52.9M [00:01<00:00, 28.5MB/s]


In [2]:
import zipfile

# Replace with the name of the downloaded ZIP file
zip_file = "/content/celebrity-face-image-dataset.zip"

# Unzip the dataset
with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall("/content/")  # Extract to the "dataset" directory

In [3]:
import os
import cv2
import dlib
import numpy as np

# Paths
root_dir = '/content/Celebrity Faces Dataset'  # Replace with the path to your dataset folder
output_dir = 'new_data_set/'  # Folder to save cropped face images
os.makedirs(output_dir, exist_ok=True)

# FACES
FACES = []
LABELS = []

detector = dlib.get_frontal_face_detector()

for subdir, _, files in os.walk(root_dir):
    processed_count = 0  # Counter for images processed in this folder
    for file in files:
        if processed_count >= 20:  # Stop if we've processed 20 images in this folder
            break
        if file.lower().endswith(('.jpg', '.jpeg', '.png')):
            image_path = os.path.join(subdir, file)
            img = cv2.imread(image_path)

            if img is None:
                continue

            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            # Detect faces
            faces = detector(img)

            for i, rect in enumerate(faces):
                x, y, width, height = rect.left(), rect.top(), rect.width(), rect.height()
                if (y + height <= img.shape[0] and x + width <= img.shape[1] and x >= 0 and y >= 0 and width > 0 and height > 0):
                    face_img = img[y:y + height, x:x + width]
                    face_img = cv2.resize(face_img, (160, 160))
                    relative_path = os.path.relpath(subdir, root_dir)  # Get relative path
                    output_subdir = os.path.join(output_dir, relative_path)  # Replicate structure
                    os.makedirs(output_subdir, exist_ok=True)

                    # Save the cropped face image
                    FACES.append(face_img)
                    LABELS.append(relative_path)
                    output_path = os.path.join(output_subdir, f"{os.path.splitext(file)[0]}_face_{i + 1}.jpg")
                    cv2.imwrite(output_path, face_img)
                    print(f"Saved face {i + 1} from {image_path} to {output_path}")

                    processed_count += 1  # Increment processed count
                    if processed_count >= 20:  # Break after saving the 20th image
                        break

print(LABELS)

FACES = np.array(FACES)
LABELS = np.array(LABELS)

print(LABELS)


Saved face 1 from /content/Celebrity Faces Dataset/Leonardo DiCaprio/037_fe4d2d8f.jpg to new_data_set/Leonardo DiCaprio/037_fe4d2d8f_face_1.jpg
Saved face 1 from /content/Celebrity Faces Dataset/Leonardo DiCaprio/100_ec9fe97f.jpg to new_data_set/Leonardo DiCaprio/100_ec9fe97f_face_1.jpg
Saved face 1 from /content/Celebrity Faces Dataset/Leonardo DiCaprio/006_30010640.jpg to new_data_set/Leonardo DiCaprio/006_30010640_face_1.jpg
Saved face 1 from /content/Celebrity Faces Dataset/Leonardo DiCaprio/087_1382b266.jpg to new_data_set/Leonardo DiCaprio/087_1382b266_face_1.jpg
Saved face 1 from /content/Celebrity Faces Dataset/Leonardo DiCaprio/091_e39f525d.jpg to new_data_set/Leonardo DiCaprio/091_e39f525d_face_1.jpg
Saved face 1 from /content/Celebrity Faces Dataset/Leonardo DiCaprio/022_66e94346.jpg to new_data_set/Leonardo DiCaprio/022_66e94346_face_1.jpg
Saved face 1 from /content/Celebrity Faces Dataset/Leonardo DiCaprio/059_1027f3c2.jpg to new_data_set/Leonardo DiCaprio/059_1027f3c2_fac

In [4]:
!pip install keras-facenet

  Preparing metadata (setup.py) ... done
  Created wheel for keras-facenet: filename=keras_facenet-0.3.2-py3-none-any.whl size=10368 sha256=8f98aad5df06bea80c18e03020daa9238babd797d8bc724025c8a346e67d5684
  Stored in directory: /root/.cache/pip/wheels/1d/d8/a9/85cf04ea29321d2afcb82c0caaafdca9195385f9d68cbc7185
Successfully built keras-facenet


In [10]:
from keras_facenet import FaceNet
embedder = FaceNet()

def get_embedding(face_img):
    # scale pixel values
    face_img = face_img.astype('float32')
    # standardize pixel values across channels (global)
    face_img = np.expand_dims(face_img, axis=0)  # Changed np.expand_dim to np.expand_dims
    yhat = embedder.embeddings(face_img)
    return yhat[0]

EMBEDDED_FACES = []

for img in FACES:
    EMBEDDED_FACES.append(get_embedding(img))

EMBEDDED_X = np.array(EMBEDDED_FACES)



1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
1/1 

In [11]:
from sklearn.preprocessing  import LabelEncoder
le = LabelEncoder()
Y = le.fit_transform(LABELS)

In [25]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC

X_train , X_test , Y_train , Y_test  = train_test_split(EMBEDDED_FACES , Y , shuffle =True , test_size=0.3 , random_state=42 )

In [26]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

model = SVC(kernel='linear', probability=True)
model.fit(X_train, Y_train)
ypreads_train = model.predict(X_train)
ypreds_test = model.predict(X_test)

In [27]:
from sklearn.metrics import accuracy_score
print(accuracy_score(Y_train, ypreads_train))
print(accuracy_score(Y_test, ypreds_test))

1.0
0.9803921568627451


In [31]:
t_im = cv2.imread('/content/chakib.jpeg')
t_im =  cv2.cvtColor(t_im, cv2.COLOR_BGR2RGB)

faces = detector(t_im)

for i, rect in enumerate(faces):
    x, y, width, height = rect.left(), rect.top(), rect.width(), rect.height()
    if (y + height <= img.shape[0] and x + width <= img.shape[1] and x >= 0 and y >= 0 and width > 0 and height > 0):
      face_img = img[y:y + height, x:x + width]
      face_img = cv2.resize(face_img, (160, 160))
      relative_path = os.path.relpath(subdir, root_dir)  # Get relative path
      output_subdir = os.path.join(output_dir, relative_path)  # Replicate structure
      os.makedirs(output_subdir, exist_ok=True)
test_im =  get_embedding(t_im)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 186ms/step


In [32]:
test_im = [test_im]
ypreds = model.predict(test_im)
print(ypreds)
le.inverse_transform(ypreds)

[2]


array(['Denzel Washington'], dtype='<U18')